In [1]:
import Pkg
Pkg.activate("..")

  Activating project at `~/research/xaqlab/ergm.jl`


# Fitted Edge-Triangle Model

In [2]:
using ergm.sampler
using ergm.spaces
using ergm.stats
using ergm.models
using ergm.inference
using ergm.optim
using LinearAlgebra
using Random

n = 30
stats = DeltaStats(
    function(G)
        p = sum(G.adjacency) / (n * (n - 1))
        [
            p,
            tr(G.adjacency ^ 3) / (n * (n - 1) * (n - 2)) - p ^ 3
        ]
    end,
    function(G, s, u)
        i, x = u
        p = s[1]
        dp = (x - G[i]) / (n * (n - 1))
        s + [
            dp,
            3 * (x - G[i]) * dot(G.adjacency[i[2], :], G.adjacency[:, i[1]]) / (n * (n - 1) * (n - 2)) - (p + dp) ^ 3 + p ^ 3
        ]
    end
)

target_Es = [1e-2, 3e-6]
G0 = DiGraph(convert(Matrix{Bool}, rand(Float64, (n, n)) .< 0.1))
θ0 = [-3916.7, 1e4]
m = ExponentialFamily(stats, θ0)
samp = ParallelGibbsSampler(
    G0,
    m,
    10,
    10,
    Threads.nthreads()
);

In [12]:
_, ss = sample(samp, 10000)

(Any[DiGraph(Bool[0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0], 30, 0), DiGraph(Bool[0 0 … 0 0; 1 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0], 30, 0), DiGraph(Bool[0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0], 30, 0), DiGraph(Bool[0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0], 30, 0), DiGraph(Bool[0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0], 30, 0), DiGraph(Bool[0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0], 30, 0), DiGraph(Bool[0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0], 30, 0), DiGraph(Bool[0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0], 30, 0), DiGraph(Bool[0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0], 30, 0), DiGraph(Bool[0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0], 30, 0)  …  DiGraph(Bool[0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0], 30, 0), DiGraph(Bool[0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0], 30, 0), DiGraph(Bool[0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0], 30, 0), DiGraph(Bool[0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0], 30, 0), DiGraph(Bool

# Platinum Data

In [4]:
using ergm.platinum

G = load_graph()

PyObject <networkx.classes.multidigraph.MultiDiGraph object at 0x7f0b772d5090>

In [ ]:
Hs = map(nx_to_digraph, subsample(G, n, 10000))

In [13]:
ss_p = hcat(map(H -> get_stats(stats, H), Hs)...)

2×10000 Matrix{Float64}:
  0.0206897    0.0126437    0.00689655  …   0.0183908   0.0252874
 -8.85645e-6  -2.02125e-6  -3.28017e-7     -6.22017e-6  0.000106983

# Plots

In [41]:
using Plots

p = histogram(hcat(ss[1, :], ss_p[1, :]), labels=["directed edge-triangle ERGM" "Platinum connectome"], dpi=300)
xlabel!("edge density statistic")
ylabel!("count")
savefig("310_figs/marginal.png")